# 引用包

In [ ]:
# encoding=utf-8
import os.path as osp
import os
import copy
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear
from sklearn.metrics import average_precision_score, roc_auc_score
from torch_geometric.data import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.datasets import ICEWS18
from torch_geometric.nn import TGNMemory, TransformerConv
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.models.tgn import (LastNeighborLoader, IdentityMessage, MeanAggregator,
                                           LastAggregator)
from torch_geometric import *
from torch_geometric.utils import negative_sampling
from tqdm import tqdm
import networkx as nx
import numpy as np
import math
import copy
import re
import time
import json
import pandas as pd
from random import choice
import gc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# msg的特征采取    [src_node_feature,edge_attr,dst_node_feature]的格式

# compute the best partition 计算出最佳的社区划分
import datetime
import community as community_louvain

import xxhash
# 查找edge向量所对应的下标
def tensor_find(t,x):
    t_np=t.numpy()
    idx=np.argwhere(t_np==x)
    return idx[0][0]+1


def std(t):
    t = np.array(t)
    return np.std(t)


def var(t):
    t = np.array(t)
    return np.var(t)


def mean(t):
    t = np.array(t)
    return np.mean(t)

def hashgen(l):
    """Generate a single hash value from a list. @l is a list of
    string values, which can be properties of a node/edge. This
    function returns a single hashed integer value."""
    hasher = xxhash.xxh64()
    for e in l:
        hasher.update(e)
    return hasher.intdigest()

# 单独计算出每条边的loss值来分析出 具体的异常行为
def cal_pos_edges_loss(link_pred_ratio):
    loss=[]
    for i in link_pred_ratio:
        loss.append(criterion(i,torch.ones(1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_multiclass(link_pred_ratio,labels):
    loss=[] 
    for i in range(len(link_pred_ratio)):
        loss.append(criterion(link_pred_ratio[i].reshape(1,-1),labels[i].reshape(-1)))
    return torch.tensor(loss)

In [ ]:
def cal_pos_edges_loss_autoencoder(decoded,msg):
    loss=[] 
    for i in range(len(decoded)):
        loss.append(criterion(decoded[i].reshape(-1),msg[i].reshape(-1)))
    return torch.tensor(loss)

In [ ]:
# 时间戳转换函数

from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
def ns_time_to_datetime(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    dt = datetime.fromtimestamp(int(ns) // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s


def ns_time_to_datetime_US(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(ns) // 1000000000, tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def time_to_datetime_US(s):
    """
    :param ns: int 秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(s), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    return s

def datetime_to_ns_time(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    return timeStamp

def datetime_to_ns_time_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp * 1000000000
    return int(timeStamp)

def datetime_to_timestamp_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp
    return int(timeStamp)


In [ ]:
# 查找edge向量所对应的下标
def tensor_find(t,x):
    t_np=t.numpy()
    idx=np.argwhere(t_np==x)
    return idx[0][0]+1      

In [ ]:
rel2id={1: 'EVENT_CONNECT',
 'EVENT_CONNECT': 1,
 2: 'EVENT_EXECUTE',
 'EVENT_EXECUTE': 2,
 3: 'EVENT_OPEN',
 'EVENT_OPEN': 3,
 4: 'EVENT_READ',
 'EVENT_READ': 4,
 5: 'EVENT_RECVFROM',
 'EVENT_RECVFROM': 5,
 6: 'EVENT_RECVMSG',
 'EVENT_RECVMSG': 6,
 7: 'EVENT_SENDMSG',
 'EVENT_SENDMSG': 7,
 8: 'EVENT_SENDTO',
 'EVENT_SENDTO': 8,
 9: 'EVENT_WRITE',
 'EVENT_WRITE': 9}

## 连接数据库

In [2]:
# 连接postgessql 数据库，将数据存入数据库中
import psycopg2

from psycopg2 import extras as ex
connect = psycopg2.connect(database = 'tc_theia_dataset_db',
                           user = 'postgres',
                           password = 'Ssys8.21',
                           port = '5432'#一般是5432
                          )



# 创建一个cursor来执行数据库的操作
cur = connect.cursor()

In [3]:
   
# 构建nodeid 与msg之间的构建  创建字典变量 nodeid2msg
sql="select * from node2id ORDER BY index_id;"
cur.execute(sql)
rows = cur.fetchall()

nodeid2msg={}  # 可以实现 nodeid 转 msg      node hash 转 nodeid
for i in rows:
    nodeid2msg[i[0]]=i[-1]
    nodeid2msg[i[-1]]={i[1]:i[2]}  #0-828297

## 加载数据

加载对应的数据集

In [ ]:
graph_4_3=torch.load("./train_graph/***")
graph_4_4=torch.load("./train_graph/***")
graph_4_5=torch.load("./train_graph/***")
graph_4_9=torch.load("./train_graph/***")

graph_4_10=torch.load("./train_graph/***")
graph_4_11=torch.load("./train_graph/***")
graph_4_12=torch.load("./train_graph/***")
graph_4_13=torch.load("./train_graph/***")
train_data=graph_4_10

# GNN

## 配置模型

In [ ]:
max_node_num = 828398 
min_dst_idx, max_dst_idx = 0, max_node_num
neighbor_loader = LastNeighborLoader(max_node_num, size=20, device=device)

class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super(GraphAttentionEmbedding, self).__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(in_channels, out_channels, heads=8,
                                    dropout=0.0, edge_dim=edge_dim)
        self.conv2 = TransformerConv(in_channels*8, out_channels,heads=1, concat=False,
                             dropout=0.0, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        last_update.to(device)
        x = x.to(device)
        t = t.to(device)
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        x = F.relu(self.conv(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        return x

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.lin_src = Linear(in_channels, in_channels*2)
        self.lin_dst = Linear(in_channels, in_channels*2)
        
        self.lin_seq = nn.Sequential(
            
            Linear(in_channels*4, in_channels*8),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels*8, in_channels*2),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels*2, int(in_channels//2)),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(int(in_channels//2), train_data.msg.shape[1]-32)                   
        )
        
    def forward(self, z_src, z_dst):
        h = torch.cat([self.lin_src(z_src) , self.lin_dst(z_dst)],dim=-1)      
         
        h = self.lin_seq (h)
        
        return h

# 更改节点嵌入向量维度
memory_dim = time_dim = embedding_dim = 100

memory = TGNMemory(
    max_node_num,
    train_data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(train_data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=train_data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=embedding_dim).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.00005, eps=1e-08,weight_decay=0.01)


# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()

# Helper vector to map global node indices to local ones.
assoc = torch.empty(max_node_num, dtype=torch.long, device=device)


saved_nodes=set()

In [ ]:
BATCH=1024
def train(train_data):

    
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.
    saved_nodes=set()

    total_loss = 0
    total_batchs=len(train_data.t)//BATCH +1
    batch_index=0
#     print("train_before_stage_data:",train_data)
    for batch in train_data.seq_batches(batch_size=BATCH):
        start = time.perf_counter()
        optimizer.zero_grad()

        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg        
        
        n_id = torch.cat([src, pos_dst]).unique()
#         n_id = torch.cat([src, pos_dst, neg_src, neg_dst]).unique()
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)
        
        print(f"{n_id=}")
        print(f"{edge_index=}")
        print(f"{train_data.msg[e_id]=}")
      
        z = gnn(z, last_update, edge_index, train_data.t[e_id], train_data.msg[e_id])
        
        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])       

        y_pred = torch.cat([pos_out], dim=0)
        
#         y_true = torch.cat([torch.zeros(pos_out.size(0),1),torch.ones(neg_out.size(0),1)], dim=0)# 0 代表正常 1 代表异常
        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l)           
          
        y_true = torch.tensor(y_true)
        y_true=y_true.reshape(-1).to(torch.long)
        
        loss = criterion(y_pred, y_true)
        
# 原先代码中的loss计算方法
#         loss = criterion(pos_out, torch.ones_like(pos_out))
#         loss += criterion(neg_out, torch.zeros_like(neg_out))

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
#         for i in range(len(src)):
#             saved_nodes.add(int(src[i]))
#             saved_nodes.add(int(pos_dst[i]))

        loss.backward()
        optimizer.step()
        memory.detach()
#         print(z.shape)
        total_loss += float(loss) * batch.num_events
        batch_index+=1
        end = time.perf_counter()
        print(f"current epoch process:{(batch_index/total_batchs):.4f}%   cost time:{(end-start):.2f}")
#     print("trained_stage_data:",train_data)
    return total_loss / train_data.num_events`````````````````


## 开始训练

In [ ]:
train_graphs=[
    graph_4_3,
    graph_4_4, 
    graph_4_5,
#     graph_4_9
]

for epoch in tqdm(range(1, 31)):
    for g in train_graphs:
        loss = train(g)
        print(f'  Epoch: {epoch:02d}, Loss: {loss:.4f}')
#     scheduler.step()
    # 将训练好的模型保存下来
model=[memory,gnn, link_pred,neighbor_loader]
torch.save(model,"./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass_without_neg_edge.pt")

# 测试环节

In [ ]:
import time 

# 检查点的时间是否设置长一些会更好呢？  比如说十分钟？
#  300000000000  是五分钟的纳秒时间段
# 测试一天的数据，每五分钟给出一个loss值  每五分钟给出一个checkpoint 的loss值
@torch.no_grad()#声明以下函数不执行梯度
def test_day_new_without_negative(inference_data,path):
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
    
#     m=torch.load("model_saved_emb100.pt")
#     memory,gnn, link_pred,neighbor_loader=m
    memory.eval()
    gnn.eval()
    link_pred.eval()
    
    memory.reset_state()  # Start with a fresh memory.  # 为什么不可以使用历史的memory呢？  应该 可以吧？
    neighbor_loader.reset_state()  # Start with an empty graph.
    
    time_with_loss={}# key: 时间段，  value： 该时间段的loss值
    total_loss = 0    
    edge_list=[]
    
    unique_nodes=torch.tensor([])
    total_edges=0
    
#     test_memory=copy.deepcopy(memory)   
#     test_gnn=copy.deepcopy(gnn)   
#     test_link_pred=copy.deepcopy(link_pred) 
#     test_neighbor_loader=copy.deepcopy(neighbor_loader)

# 记录起始的时间点

    start_time=inference_data.t[0]
    event_count=0
    
    pos_o=[]
    
    loss_list=[]
    
#     print("before merge:",train_data)

#     nique_node_count=len(torch.cat([train_data.src,train_data.dst]).unique())

    print("after merge:",inference_data)
    
    # 记录程序运行时间  评估运行效率
    start = time.perf_counter()

    for batch in inference_data.seq_batches(batch_size=BATCH):
        
        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg
        unique_nodes=torch.cat([unique_nodes,src,pos_dst]).unique()
        total_edges+=BATCH
        
       
        n_id = torch.cat([src, pos_dst]).unique()       
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        #如果memory和neighbor_loader在测试阶段都没有被更新的话，此处不需要采用gnn获得z矩阵，只需要从memory查出来对应的节点特征向量即可
        z = gnn(z, last_update, edge_index, inference_data.t[e_id], inference_data.msg[e_id])

        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])
        
        pos_o.append(pos_out)
        y_pred = torch.cat([pos_out], dim=0)
#         y_true = torch.cat(
#             [torch.ones(pos_out.size(0))], dim=0).to(torch.long)     
#         y_true=y_true.reshape(-1).to(torch.long)

        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l) 
        y_true = torch.tensor(y_true)
        y_true=y_true.reshape(-1).to(torch.long)

        # 只考虑边有没有被正确预测，对于正常行为的图而言，行为模式比较相似所以loss较低。  对于异常行为，会存在一些行为没见过，所以对这些行为预测存在的概率就地，所以loss也会高。
        loss = criterion(y_pred, y_true)

        total_loss += float(loss) * batch.num_events
     
        
# 将当前batch 发生的edge 更新到memory 和neighbor中
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
        #计算每条边的loss值
        each_edge_loss= cal_pos_edges_loss_multiclass(pos_out,y_true)
        
        for i in range(len(pos_out)):
            srcnode=int(src[i])
            dstnode=int(pos_dst[i])  
            
            srcmsg=str(nodeid2msg[srcnode]) 
            dstmsg=str(nodeid2msg[dstnode])
            t_var=int(t[i])
            edgeindex=tensor_find(msg[i][16:-16],1) # find 找出来的范围是 1-n   rel2id中的id也是1-n    
            edge_type=rel2id[edgeindex]
            loss=each_edge_loss[i]    

            temp_dic={}
            temp_dic['loss']=float(loss)
            temp_dic['srcnode']=srcnode
            temp_dic['dstnode']=dstnode
            temp_dic['srcmsg']=srcmsg
            temp_dic['dstmsg']=dstmsg
            temp_dic['edge_type']=edge_type
            temp_dic['time']=t_var
            
            # 先不考虑与socket 节点 当找出可疑的进程与文件之后再将socket找出
#             if "netflow" in srcmsg or "netflow" in dstmsg:
#                 temp_dic['loss']=0
            edge_list.append(temp_dic)
        
        event_count+=len(batch.src)
        if t[-1]>start_time+60000000000*15:
            # 此处为一个checkpoint  输出loss值 并且清空全局的loss值  保存处理过的节点
#             loss=total_loss/event_count
            time_interval=ns_time_to_datetime_US(start_time)+"~"+ns_time_to_datetime_US(t[-1])

            end = time.perf_counter()
            time_with_loss[time_interval]={'loss':loss,
                                
                                          'nodes_count':len(unique_nodes),
                                          'total_edges':total_edges,
                                          'costed_time':(end-start)}
            
            
            log=open(path+"/"+time_interval+".txt",'w')
            # 减去train data中没有被训练好的
            
            for e in edge_list: 
#                 temp_key=e['srcmsg']+e['dstmsg']+e['edge_type']
#                 if temp_key in train_edge_set:      
# #                     e['loss']=(e['loss']-train_edge_set[temp_key]) if e['loss']>=train_edge_set[temp_key] else 0  
# #                     e['loss']=abs(e['loss']-train_edge_set[temp_key])
                    
#                     e['modified']=True
#                 else:
#                     e['modified']=False
                loss+=e['loss']

            loss=loss/event_count   
            print(f'Time: {time_interval}, Loss: {loss:.4f}, Nodes_count: {len(unique_nodes)}, Cost Time: {(end-start):.2f}s')
            edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True)   # 按照loss 值排序  或者按照edge的时间顺序排列
            for e in edge_list: 
                log.write(str(e))
                log.write("\n") 
            event_count=0
            total_loss=0
            loss=0
            start_time=t[-1]
            log.close()
            edge_list.clear()
            
 
    return time_with_loss


# 测试4-9 ~ 4-12

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_3=test_day_new(graph_4_3,"graph_4_3")

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_4=test_day_new(graph_4_4,"graph_4_4")

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_5=test_day_new(graph_4_5,"graph_4_5")

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_9=test_day_new(graph_4_9,"graph_4_9")

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_10=test_day_new(graph_4_10,"graph_4_10")

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_11=test_day_new(graph_4_11,"graph_4_11")

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_12=test_day_new(graph_4_12,"graph_4_12")

## 计算IDF函数 异常分数函数

In [ ]:
def cal_train_IDF(find_str,file_list):
    include_count=0
    for f_path in (file_list):
        f=open(f_path)
        if find_str in f.read():
            include_count+=1             
    IDF=math.log(len(file_list)/(include_count+1))
    return IDF


def cal_IDF(find_str,file_path,file_list):
    file_list=os.listdir(file_path)
    include_count=0
    different_neighbor=set()#至少有两个 因此要减去2
    for f_path in (file_list):
        f=open(file_path+f_path)
        if find_str in f.read():
            include_count+=1
#         add=True
#         for line in f:
            
#             if find_str in line:
# #                 print(line)
#                 if add:
#                     include_count+=1
#                     add=False
#                 l=line.strip()
#                 jdata=eval(l)
#                 different_neighbor.add(jdata['srcmsg'])
#                 different_neighbor.add(jdata['dstmsg'])
                
                
    IDF=math.log(len(file_list)/(include_count+1))
    
    return IDF,1

def cal_IDF_by_file_in_mem(find_str,file_list):

    include_count=0
    different_neighbor=set()#至少有两个 因此要减去2
    for f_path in (file_list):
        f=open(file_path+f_path)
        if find_str in f.read():
            include_count+=1
#         add=True
#         for line in f:
            
#             if find_str in line:
# #                 print(line)
#                 if add:
#                     include_count+=1
#                     add=False
#                 l=line.strip()
#                 jdata=eval(l)
#                 different_neighbor.add(jdata['srcmsg'])
#                 different_neighbor.add(jdata['dstmsg'])
                
                
    IDF=math.log(len(file_list)/(include_count+1))
    
    return IDF,1

def cal_redundant(find_str,edge_list):
    
    different_neighbor=set()#至少有两个 因此要减去2
    for e in edge_list:
        if find_str in str(e):
            different_neighbor.add(e[0])
            different_neighbor.add(e[1])
    return len(different_neighbor)-2

def cal_anomaly_loss(loss_list,edge_list,file_path):
    
    if len(loss_list)!=len(edge_list):
        print("error!")
        return 0
    count=0
    loss_sum=0
    loss_std=std(loss_list)
    loss_mean=mean(loss_list)
    edge_set=set()
    node_set=set()
    node2redundant={}
    
    thr=loss_mean+1*loss_std
#     thr=0 # 因为训练的时候采用的就是整个batch均值的方式进行训练
    print("thr:",thr)
  
    for i in range(len(loss_list)):
        if loss_list[i]>thr:
            count+=1
            src_node=edge_list[i][0]
            dst_node=edge_list[i][1]
          
            loss_sum+=loss_list[i]
    
            node_set.add(src_node)
            node_set.add(dst_node)
            edge_set.add(edge_list[i][0]+edge_list[i][1])
    return count, loss_sum/(count+0.00000000001),node_set,edge_set
#     return count, count/len(loss_list)

## 以15min为间隔计算IDF

使用之前的IDF计算方法

# 窗口关系建立函数

这里有两个窗口建立函数，一个用于4-10号 一个用于4-12  有一点点小的差异。

In [ ]:
def is_include_key_word(s):
    keywords=[
         'netflow',        
        '/dev/pts',
        'salt-minion.log',
        'null',
        'usr',
         'proc',
        'firefox',
        'tmp',
        'thunderbird',
        'bin/',
        '/data/replay_logdb',
        '/stat',
        '/boot',
        'qt-opensource-linux-x64',
        '/eraseme'
        '675',
      ]
    flag=False
    for i in keywords:
        if i in s:
            flag=True
    return flag


def cal_set_rel(s1,s2,file_list):
    new_s=s1 & s2
    count=0
    for i in new_s:
#     jdata=json.loads(i)
       if is_include_key_word(i) is not True:
        
#         'netflow' not in i
#         and 'usr' not in i and 'var' not in i
            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                IDF=math.log(len(file_list)/(1))
                
            if i in node_IDF_3.keys():
                IDF3=node_IDF_3[i]
            else:
                IDF3=math.log((files_count_4_3)/(1))    
            
#             print(IDF)
            if (IDF+IDF3)>5 :
                print("node:",i," IDF:",IDF)
                count+=1
    return count

In [ ]:

def is_include_key_word_bak(s):
    keywords=[
         'netflow',
        'null',
        '/dev/pts',
        'salt-minion.log',
        '675',
        'usr',
         'proc',
        '/.cache/mozilla/',
        'tmp',
        'thunderbird',
        '/bin/',
        '/sbin/sysctl',
#         '/data/replay_logdb/',
        '/home/admin/eraseme',
      ]
    flag=False
    for i in keywords:
        if i in s:
            flag=True
    return flag


def cal_set_rel_bak(s1,s2,file_list):
    new_s=s1 & s2
    count=0
    for i in new_s:
#     jdata=json.loads(i)
        if is_include_key_word_bak(i) is not True:
            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                IDF=math.log(len(file_list)/(1))         

            if (IDF)>math.log(len(file_list)*0.9/(1))  :
                print("node:",i," IDF:",IDF)
                count+=1
    return count

# label生成

In [ ]:
labels={}

    
filelist = os.listdir("graph_4_10_without_neg_edge")
for f in filelist:
    labels["graph_4_10_without_neg_edge/"+f]=0

filelist = os.listdir("graph_4_11_without_neg_edge")
for f in filelist:
    labels["graph_4_11_without_neg_edge/"+f]=0
    
filelist = os.listdir("graph_4_12_without_neg_edge")
for f in filelist:
    labels["graph_4_12_without_neg_edge/"+f]=0


In [ ]:
attack_list=[
    
'graph_4_10_without_neg_edge/2018-04-10 13:31:14.548738409~2018-04-10 13:46:36.161065223.txt', 
 'graph_4_10_without_neg_edge/2018-04-10 14:02:17.001271389~2018-04-10 14:17:34.001373488.txt', 
 'graph_4_10_without_neg_edge/2018-04-10 14:17:34.001373488~2018-04-10 14:33:18.350772859.txt', 
 'graph_4_10_without_neg_edge/2018-04-10 14:33:18.350772859~2018-04-10 14:48:47.320442910.txt',
'graph_4_10_without_neg_edge/2018-04-10 14:48:47.320442910~2018-04-10 15:03:54.307022037.txt', 
 
 'graph_4_12_without_neg_edge/2018-04-12 12:39:06.592749611~2018-04-12 12:54:44.001952179.txt', 
 'graph_4_12_without_neg_edge/2018-04-12 12:54:44.001952179~2018-04-12 13:09:55.027159876.txt', 
 'graph_4_12_without_neg_edge/2018-04-12 13:09:55.027159876~2018-04-12 13:25:06.588443090.txt', 
 'graph_4_12_without_neg_edge/2018-04-12 13:25:06.588443090~2018-04-12 13:40:07.178285911.txt'
]

for i in attack_list:
    labels[i]=1

# 异常检测

## 4-10异常检测

In [ ]:
# node_IDF_3=torch.load("node_IDF_4_3")
node_IDF=torch.load("node_IDF_4_3")
file_list=[]

file_path="graph_4_10_without_neg_edge/"
file_l=os.listdir("graph_4_10_without_neg_edge/")
for i in file_l:
    file_list.append(file_path+i)
    
    
    
# node_IDF_410=torch.load("node_IDF_4_10")
# node_IDF=torch.load("node_IDF_4_12")
y_data_4_10=[]
df_list_4_10=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}



file_path_list=[]


file_path="graph_4_10_without_neg_edge/"
file_l=os.listdir("graph_4_10_without_neg_edge/")
for i in file_l:
    file_path_list.append(file_path+i)

# file_path="graph_4_12/"
# file_l=os.listdir("graph_4_12/")
# for i in file_l:
#     file_path_list.append(file_path+i)


index_count=0
for f_path in (file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_10.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_10_without_neg_edge/")
    current_tw={}
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
        if added_que_flag:
            break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))
#     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

In [ ]:

pred_label={}

# files = os.listdir("graph_4_9")
# for f in files:
#     pred_label["graph_4_9/"+f]=0

files = os.listdir("graph_4_10_without_neg_edge")
for f in files:
    pred_label["graph_4_10_without_neg_edge/"+f]=0

files = os.listdir("graph_4_11_without_neg_edge")
for f in files:
    pred_label["graph_4_11_without_neg_edge/"+f]=0

files = os.listdir("graph_4_12_without_neg_edge")
for f in files:
    pred_label["graph_4_12_without_neg_edge/"+f]=0


for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)

    if loss_count>10:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

## 4-12异常检测

In [ ]:
node_IDF=torch.load("node_IDF_4_12") # 单独计算下4-12的idf值
node_IDF_3=torch.load("node_IDF_4_3")
file_list=[]

file_path="graph_4_12_without_neg_edge/"
file_l=os.listdir("graph_4_12_without_neg_edge/")
for i in file_l:
    file_list.append(file_path+i)
    
#以下变量名对检测结果影响不大，所以当初就没改   
y_data_4_10=[]
df_list_4_10=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}



file_path_list=[]


file_path="graph_4_12_without_neg_edge/"
file_l=os.listdir("graph_4_12_without_neg_edge/")
for i in file_l:
    file_path_list.append(file_path+i)


index_count=0
for f_path in (file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_10.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_12_without_neg_edge/")
    current_tw={}
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'],file_l)!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
        if added_que_flag:
            break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
    index_count+=1
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))


In [ ]:
for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

# 统计该场景的attack edges数量

In [4]:
def keyword_hit(line):
    attack_nodes=[
            '/home/admin/clean',
            '/dev/glx_alsa_675',
            '/home/admin/profile',
#             '/var/log/mail',  # 添加以后会将端口扫描的日志加入，这部分的日志会十分庞大。
            '/tmp/memtrace.so',
            '/var/log/xdev',
             '/var/log/wdev',
            'gtcache',
#             'firefox',
        '161.116.88.72',
        '146.153.68.151',
        '145.199.103.57',
        '61.130.69.232',
        '5.214.163.155',
        '104.228.117.212',
        '141.43.176.203',
        '7.149.198.40',
        '5.214.163.155',
        '149.52.198.23',
        ]
    flag=False
    for i in attack_nodes:
        if i in line:
            flag=True
            break
    return flag



files=[]
temp_file=[
         '2018-04-10 13:33:14.002087332~2018-04-10 13:48:35.001709448.txt',
 '2018-04-10 14:19:04.002022895~2018-04-10 14:34:39.687914951.txt',
 '2018-04-10 14:34:39.687914951~2018-04-10 14:49:52.239141721.txt',
 '2018-04-10 14:49:52.239141721~2018-04-10 15:05:26.721606494.txt',
]
for f in temp_file:
    files.append("./graph_4_10_bak/graph_4_10/"+f)
    
    
temp_file=[
 '2018-04-12 12:37:14.001187512~2018-04-12 12:52:38.001955351.txt', 
 '2018-04-12 12:52:38.001955351~2018-04-12 13:07:55.787879623.txt', 
 '2018-04-12 13:07:55.787879623~2018-04-12 13:23:35.144633810.txt',
 '2018-04-12 13:23:35.144633810~2018-04-12 13:38:35.600938143.txt',
]    
for f in temp_file:
    files.append("./graph_4_12_bak/graph_4_12/"+f)

# 可视化分析模块

In [ ]:
replace_dic={
    '/run/shm/':'/run/shm/*',
#     '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/':'/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/*',
   '/home/admin/.cache/mozilla/firefox/':'/home/admin/.cache/mozilla/firefox/*',
    '/home/admin/.mozilla/firefox':'/home/admin/.mozilla/firefox*',    
    '/data/replay_logdb/':'/data/replay_logdb/*', 
    '/home/admin/.local/share/applications/':'/home/admin/.local/share/applications/*', 
    '/usr/share/applications/':'/usr/share/applications/*', 
    '/lib/x86_64-linux-gnu/':'/lib/x86_64-linux-gnu/*',     
    '/proc/':'/proc/*', 
     '/stat':'*/stat', 
    '/etc/bash_completion.d/':'/etc/bash_completion.d/*', 
    '/usr/bin/python2.7':'/usr/bin/python2.7/*', 
     '/usr/lib/python2.7':'/usr/lib/python2.7/*', 


}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    return path_name

In [ ]:
replace_dic_draw={
    '/run/shm/':'/run/shm/*',
   '/home/admin/.cache/mozilla/firefox/':'/home/admin/.cache/mozilla/firefox/*',
    '/home/admin/.mozilla/firefox':'/home/admin/.mozilla/firefox*',    
    '/data/replay_logdb/':'/data/replay_logdb/*', 
    '/home/admin/.local/share/applications/':'/home/admin/.local/share/applications/*', 
    
    '/usr/share/applications/':'/usr/share/applications/*', 
    '/lib/x86_64-linux-gnu/':'/lib/x86_64-linux-gnu/*',     
    '/proc/':'/proc/*', 
     '/stat':'*/stat', 
    '/etc/bash_completion.d/':'/etc/bash_completion.d/*', 
    '/usr/bin/python2.7':'/usr/bin/python2.7/*', 
     '/usr/lib/python2.7':'/usr/lib/python2.7/*', 
    

}

def replace_path_name_draw(path_name):
    for i in replace_dic_draw:
        if i in path_name:
            return replace_dic_draw[i]
    return path_name

In [ ]:
def attack_edge_flag(msg):
    attack_edge_type=[
        '/home/admin/clean',
        '/dev/glx_alsa_675',
        '/home/admin/profile',
          '/var/log/xdev',
    '/etc/passwd',
    '161.116.88.72',
    '146.153.68.151',
        '/var/log/mail',
        '/tmp/memtrace.so',
#         '/tmp',
        '/var/log/xdev',
         '/var/log/wdev',
        'gtcache',
        'firefox',
#         '/var/log',
    ]
    flag=False
    for i in attack_edge_type:
        if i in msg:
            flag=True
            break
    return flag